<a href="https://colab.research.google.com/github/sarisppp/PJ63_CE18/blob/master/15_11_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


ทำการ import Envoloment แล้ว โหลดข้อมูล

In [5]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler
%matplotlib inline


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
used_features = ['Timestamp','SET50.Close','EMAV(34)','RSI(14)','EMAV(RSI,13)','MACD(13,5,5)','Signal','MSTO%K','MSTO%D','DI+','DI-','ADX']
data1 = pd.read_excel("/content/drive/My Drive/Data/new/Set50_130919_200820_1.xlsx",usecols =used_features)
data2 = pd.read_excel("/content/drive/My Drive/Data/new/Set50_210829_191020_2.xlsx",usecols =used_features)
data3 = pd.read_excel("/content/drive/My Drive/Data/new/Set50_201020_041120_3.xlsx",usecols =used_features)
data4 = pd.read_excel("/content/drive/My Drive/Data/new/Set50_051120_060121_4.xlsx",usecols =used_features)
data = pd.concat([data1,data2,data3,data4],ignore_index=True)
data["Timestamp"] = pd.to_datetime(data['Timestamp']) #ทำวันที่
data.set_index("Timestamp",inplace=True)
#data.reset_index("Timestamp") # ยกเลิก Index Row
data=data.dropna()
data.tail(100)
#data.loc["2020-08"]


In [ ]:
data.info() # ดูข้อมูล

In [9]:
for i in ["EMAV(34)","MSTO%K","MSTO%D","RSI(14)","ADX","DI+","DI-","MACD(13,5,5)","Signal","EMAV(RSI,13)"]:
  data[i] = pd.to_numeric(data[i],errors='coerce')

In [10]:
data.isnull().sum()
data=data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 81447 entries, 2019-03-13 14:59:00 to 2021-06-01 16:46:00
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   81447 non-null  float64
 1   EMAV(34)      81447 non-null  float64
 2   MSTO%K        81447 non-null  float64
 3   MSTO%D        81447 non-null  float64
 4   RSI(14)       81447 non-null  float64
 5   ADX           81447 non-null  float64
 6   DI+           81447 non-null  float64
 7   DI-           81447 non-null  float64
 8   MACD(13,5,5)  81447 non-null  float64
 9   Signal        81447 non-null  float64
 10  EMAV(RSI,13)  81447 non-null  float64
dtypes: float64(11)
memory usage: 7.5 MB


In [ ]:
#plt.plot(data["Timestamp"].head(100))
plt.figure(figsize=(40,20))
plt.plot(data.index,data["SET50.Close"])
plt.show() 

In [11]:
def signal(data,periods=16):
  data['output']=0
  for index,row in data.iterrows():
    if row['MACD(13,5,5)']>row['Signal'] and row['RSI(14)']>row['EMAV(RSI,13)'] and row['RSI(14)']<70:
      signalPre=1
    elif row['MACD(13,5,5)']<row['Signal'] and row['RSI(14)']<row['EMAV(RSI,13)'] and row['RSI(14)']>30:
      if row['SET50.Close']<row['EMAV(34)']:
        signalPre=-1
      else:
        signalPre=0
    else:
      signalPre=0
    data.at[index,'output']= signalPre
    
     
  return data

In [12]:
def updown(data):
    data['UpHoldDown']=0
    for index,row in data.iterrows():
      if row['SET50.Close']>row['SET50.Close']:
        signalpre = 1
      elif row['SET50.Close']<row['SET50.Close']:
        signalpre = -1
      else:
        signalpre = 0
      data.at[index,'UpHoldDown']=signalpre
    
    return data

In [13]:
#เงื่อนไขใหม่
def signal2(data,periods=16):
  data['output2']=0
  for index,row in data.iterrows():
    if row['MACD(13,5,5)']>row['Signal'] and row['RSI(14)']>row['EMAV(RSI,13)'] and row['RSI(14)']<70 and row['MSTO%K']>row["MSTO%D"] and row["DI+"]>row["DI-"]:
      signalPre=1
    elif row['MACD(13,5,5)']<row['Signal'] and row['RSI(14)']<row['EMAV(RSI,13)'] and row['RSI(14)']>30 and row['MSTO%K']<row["MSTO%D"] and row["DI+"]<row["DI-"]:
      if row['SET50.Close']<row['EMAV(34)']:
        signalPre=-1
      else:
        signalPre=0
    else:
      signalPre=0
    data.at[index,'output2']= signalPre
    
     
  return data

In [14]:
data['UHD']=0
for i in range(len(data["SET50.Close"])):
  if data["SET50.Close"][i]>data["SET50.Close"][i-1]:
    signalpre = 1 
  elif data["SET50.Close"][i]<data["SET50.Close"][i-1]:
    signalpre = -1
  else:
    signalpre = 0
  data["UHD"][i]=signalpre
#data.head(40)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
data.tail(20)

,SET50.Close,EMAV(34),MSTO%K,MSTO%D,RSI(14),ADX,DI+,DI-,"MACD(13,5,5)",Signal,"EMAV(RSI,13)",UHD
Timestamp,,,,,,,,,,,,
2021-06-01 16:18:00,940.37,941.8072,92.697,90.496,46.3002,37.556,33.564,35.633,-0.1465,-0.8086,35.202,1
2021-06-01 16:19:00,940.29,941.7205,84.853,87.675,45.9098,35.035,32.452,35.714,0.0461,-0.5237,36.732,-1
2021-06-01 16:20:00,940.76,941.6656,79.242,83.458,48.6493,32.357,34.232,34.084,0.2436,-0.2679,38.434,1
2021-06-01 16:21:00,939.67,941.5516,55.080,69.269,43.1865,30.346,31.226,35.361,0.1372,-0.1329,39.113,-1
2021-06-01 16:22:00,939.20,941.4172,27.926,48.597,41.0462,29.103,29.404,39.132,-0.0197,-0.0952,39.389,-1
2021-06-01 16:23:00,939.89,941.3299,26.411,37.504,45.3298,27.341,32.269,36.530,0.0230,-0.0558,40.238,1
2021-06-01 16:24:00,939.61,941.2317,37.681,37.593,43.9347,25.273,32.946,32.650,-0.0070,-0.0395,40.766,-1
2021-06-01 16:25:00,939.23,941.1173,37.668,37.630,42.0436,23.474,30.382,29.257,-0.0962,-0.0584,40.949,-1
2021-06-01 16:26:00,939.37,941.0174,22.759,30.195,43.0167,21.703,27.370,27.120,-0.1159,-0.0775,41.244,1


In [12]:
g = ['UHD']
data.groupby(by= g)['SET50.Close'].count()

UHD
-1    39574
 0     2504
 1    39369
Name: SET50.Close, dtype: int64

In [16]:
data_new = data
data_new = signal2(data)
data_new.loc["20-08-20":"20-08-20 12"].head(20)


,SET50.Close,EMAV(34),MSTO%K,MSTO%D,RSI(14),ADX,DI+,DI-,"MACD(13,5,5)",Signal,"EMAV(RSI,13)",UHD,output2
Timestamp,,,,,,,,,,,,,
2020-08-20 09:58:00,840.14,850.7779,0.000,0.019,12.3991,36.930,4.467,77.038,-2.2307,-0.9798,39.149,-1,0
2020-08-20 09:59:00,842.24,850.2900,10.067,5.043,24.0116,40.157,7.910,67.007,-2.8386,-1.5994,36.987,1,0
2020-08-20 10:00:00,844.22,849.9431,24.641,14.842,33.0262,41.108,18.365,58.989,-2.6737,-1.9575,36.421,1,0
2020-08-20 10:01:00,845.96,849.7155,55.421,35.132,39.7865,40.668,25.442,53.316,-2.1208,-2.0119,36.902,1,0
2020-08-20 10:02:00,845.35,849.4661,91.840,63.486,38.3259,40.084,25.535,50.763,-1.8200,-1.9479,37.105,-1,0
2020-08-20 10:03:00,845.62,849.2463,85.404,74.445,39.3866,39.544,24.755,49.213,-1.5100,-1.8020,37.431,1,0
2020-08-20 10:04:00,845.78,849.0482,71.783,73.114,40.0446,39.262,23.636,50.085,-1.2305,-1.6115,37.804,1,0
2020-08-20 10:05:00,845.60,848.8512,63.391,68.252,39.5247,38.697,24.995,48.428,-1.0465,-1.4231,38.050,-1,0
2020-08-20 10:06:00,846.32,848.7065,70.248,69.250,42.7275,37.828,25.988,45.607,-0.7543,-1.2002,38.718,1,0


In [17]:
signal2(data)

,SET50.Close,EMAV(34),MSTO%K,MSTO%D,RSI(14),ADX,DI+,DI-,"MACD(13,5,5)",Signal,"EMAV(RSI,13)",UHD,output2
Timestamp,,,,,,,,,,,,,
2019-03-13 14:59:00,1081.74,1081.6691,28.800,40.430,49.0514,13.328,30.881,22.476,0.0831,0.2186,53.734,1,0
2019-03-13 15:00:00,1082.10,1081.6937,18.338,29.384,52.0825,13.515,29.443,21.429,0.0583,0.1652,53.498,1,0
2019-03-13 15:01:00,1081.04,1081.6564,9.574,19.479,43.8165,12.781,26.118,28.279,-0.1605,0.0566,52.115,-1,-1
2019-03-13 15:02:00,1082.73,1081.7177,50.868,35.174,55.8480,12.927,31.803,23.663,0.0440,0.0524,52.648,1,0
2019-03-13 15:03:00,1082.66,1081.7716,67.551,51.362,55.3196,13.061,30.835,22.943,0.1454,0.0834,53.030,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01 16:38:00,940.41,940.8151,67.857,75.653,49.7550,14.434,24.328,27.550,0.2330,0.2286,46.990,0,0
2021-06-01 16:40:00,940.41,940.7919,0.000,37.826,49.7550,13.802,24.328,27.550,0.2052,0.2208,47.385,0,0
2021-06-01 16:44:00,940.41,940.7701,0.000,18.913,49.7550,13.218,24.328,27.550,0.1796,0.2071,47.724,0,0


ตรวจสอบฟังชั่นกับราคาจริง

In [ ]:
data['UHD=OUTPUT2']=0
for i in range(len(data["UHD"])):
  if data["UHD"][i]==data["output2"][i-1]:
    s = 1 
  else:
    s = 0
  data["UHD=OUTPUT2"][i]=s

In [ ]:
data['UHD=PREDICT']=0
for i in range(len(data["UHD"])):
  if data["UHD"][i]==data["signal_predict"][i-1]:
    s = 1 
  else:
    s = 0
  data["UHD=PREDICT"][i]=s


In [ ]:
data.groupby(["UHD=OUTPUT2"])[["SET50.Close"]].count()

In [ ]:
a=0
b=0
for i in range(len(data["UHD=OUTPUT2"])):
  if data["UHD=OUTPUT2"][i]==1:
    a+=1
  else:
    b+=1
print("TRUE",a)
print("FALSE",a)

In [ ]:
data.tail(50)

In [ ]:
data.to_excel('new.xlsx') #export file

In [20]:
data2 = data.iloc[:,0:12]
X = data2.iloc[:,0:11]
y = data2["UHD"]

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier()
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2)

In [22]:
print("Total X: {}".format(len(X)))
print("Total Train: {}".format(len(X_train)))
print("Total Test: {}".format(len(X_test)))

Total X: 81447
Total Train: 65157
Total Test: 16290


In [23]:
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [24]:
y_predict = model.predict(X_test)
y_predict


array([ 1, -1,  1, ...,  1,  1,  1])

In [25]:
y_test

Timestamp
2020-05-29 12:12:00   -1
2020-12-30 12:28:00   -1
2020-09-11 14:38:00   -1
2020-02-19 11:41:00   -1
2020-07-17 16:11:00    1
                      ..
2020-02-09 11:29:00   -1
2019-11-09 14:48:00   -1
2019-12-18 15:15:00   -1
2019-12-09 15:05:00    1
2019-11-28 11:05:00    1
Name: UHD, Length: 16290, dtype: int64

In [26]:
print("Score = {}".format(accuracy_score(y_test,y_predict)*100))

Score = 72.49232658072438


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=0)
print(X_train.info())
print("=====================================")
print(y_train.head())
RandomForest = RandomForestClassifier(n_estimators=100,verbose=2)
RandomForest.fit(X_train, y_train)

In [29]:
y_predict=RandomForest.predict(X_test)

from sklearn.metrics import classification_report 
report = classification_report(y_test,y_predict)
print(report)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

          -1       0.80      0.80      0.80      7951
           0       0.95      0.59      0.73       496
           1       0.79      0.81      0.80      7843

    accuracy                           0.80     16290
   macro avg       0.85      0.73      0.78     16290
weighted avg       0.80      0.80      0.80     16290



[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [ ]:
xx = "SET50.Close","EMAV(34)","MSTO%K","MSTO%D","RSI(14)","ADX","DI+","DI-","MACD(13,5,5)","Signal","EMAV(RSI,13)"
yy = "UHD"
import pydotplus
from sklearn.tree import export_graphviz
from IPython.display import Image

dot_data = export_graphviz(model,out_file="tree",
                           feature_names=xx,
                           class_names=yy,
                           proportion=True,
                           filled=True,rounded=True,
                            impurity=False)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
export_graphviz(model,
                out_file='tree.dot',
                feature_names=encoded_data.columns,
                proportion=True,
                filled=True,
                impurity=False)